# Projenin başlığı

#### Öğrenci ad(lar)ı

Abstract of the project comes here.

## İçerik

[Problem](#problem)   
[Verinin Anlaşılması](#data_understanding)   
[Verilerin Hazırlanması](#data_preparation)   
[Modelleme](#modeling)   
[Değerlendirme](#evaluation)   
[Referanslar](#references)   


## Problem <a class="anchor" id="problem"></a>

Describe the problem here. What are the questions you are trying to solve?

## Verinin Anlaşılması<a class="anchor" id="data_understanding"></a>

In [2]:
#Kütüphanlerin yüklenmesi

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Kullanıcıların dağılımlarını göstermek için bir dünya haritası oluşturulması
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

# Tarih için Zaman Serisi Verilerini Kullanma
import datetime as dt
from datetime import timedelta

# Makine Ögrenmesi kütüphanleri
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score

import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [3]:
# Verilerin yüklenmesi
train_data = pd.read_csv('data/covid19-global-forecasting-week-4/train.csv')
test_data = pd.read_csv('data/covid19-global-forecasting-week-4/test.csv')
countries_data = pd.read_csv('data/countries_useful_features.csv')

Hangi veri setini kullanacaksınız? Nereden alacaksın? Verilerinizi açıklayın.

2019 Johns Coron Üniversitesi Sistem Bilimi ve Mühendisliği Merkezi'nden (JHU CSSE) elde edilen yeni Coronavirus verileri [4]. 19 Nisan 2020 itibariyle 210 ülkede 2 milyondan fazla vaka bildirilmiştir.
Bu verilerin sayısı gün geçtikçe artmaktadır. 

Veriler Kaggle tarafından eğitim verilerine ve test verilerine ayrılmıştır. Tren verilerindeki özellikler (~ 27232 veri) aşağıdaki gibidir:

    
<ol>
    <li>Kimlik [ID]: Benzersiz Tanımlayıcı</li>
    <li>Province_State [Dize]: İl / eyalet ör. ABD'de “Washington”</li>
    <li>Country_Region [String]: Verilerin sağlandığı ülke</li>
    <li>Tarih [String]: 22 Ocak 20 - Bugün arasındaki günler</li>
    <li>ConfirmedCases [Decimal]: Onaylanan vakaların sayısı</li>
    <li>Ölümler [Ondalık]: Kayıtlı ölüm sayısı</li>
<ol>

In [4]:
train_data.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [5]:
test_data.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-04-02
1,2,NaN,Afghanistan,2020-04-03
2,3,NaN,Afghanistan,2020-04-04
3,4,NaN,Afghanistan,2020-04-05
4,5,NaN,Afghanistan,2020-04-06


In [6]:
countries_data.head()

,Country_Region,Population_Size,Tourism,Date_FirstFatality,Date_FirstConfirmedCase,Latitude,Longtitude,Mean_Age,Lockdown_Date,Lockdown_Type,Country_Code
0,Afghanistan,37172386,14000,2020-03-23,2020-02-25,33.939110,67.709953,17.3,2020-03-24,Full,AFG
1,Albania,2866376,5340000,2020-03-12,2020-03-10,41.153332,20.168331,36.2,2020-03-08,Full,ALB
2,Algeria,42228429,2657000,2020-03-13,2020-02-26,28.033886,1.659626,27.5,2020-03-24,Full,DZA
3,Andorra,77006,3042000,2020-03-23,2020-03-03,42.546245,1.601554,37.0,2020-03-16,Full,AND
4,Angola,30809762,218000,2020-03-30,2020-03-21,-11.202692,17.873887,16.4,NaN,NaN,AGO


In [8]:
train_data.describe(include='all')

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
count,33804.00000,14364,33804,33804,33804.000000,33804.000000
unique,NaN,133,184,108,NaN,NaN
top,NaN,Fujian,US,2020-02-17,NaN,NaN
freq,NaN,108,5832,313,NaN,NaN
mean,17838.50000,NaN,NaN,NaN,3037.990267,198.782452
std,10300.65312,NaN,NaN,NaN,16629.397127,1607.179156
min,1.00000,NaN,NaN,NaN,0.000000,0.000000
25%,8919.75000,NaN,NaN,NaN,0.000000,0.000000
50%,17838.50000,NaN,NaN,NaN,14.000000,0.000000
75%,26757.25000,NaN,NaN,NaN,414.250000,6.000000


In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33804 entries, 0 to 33803
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              33804 non-null  int64  
 1   Province_State  14364 non-null  object 
 2   Country_Region  33804 non-null  object 
 3   Date            33804 non-null  object 
 4   ConfirmedCases  33804 non-null  float64
 5   Fatalities      33804 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.5+ MB


## Verilerin Hazırlanması <a class="anchor" id="data_preparation"></a>

Explain what kind of data transformations, feature selection and/or engineering you will perform.

## Modelleme <a class="anchor" id="modeling"></a>

Which model will be used? Why? What parameters?

## Değerlendirme <a class="anchor" id="evaluation"></a>

Evaluate your model. Provide results, tables, charts, etc.

## Referanslar<a class="anchor" id="references"></a>

List all the sources you used during your work.
This includes jupyter notebooks you found on Internet.
Remeber, your work may not be an original one. However, this document must be genuine. Copy and paste kind of deliveries will be punished badly.


**Disclaimer!** <font color='grey'>This notebook was prepared by <student name(s)> as a term project for the *BBM469 - Data Intensive Applications Laboratory* class. The notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided as it is a student work.

If you think there is any copyright violation, please let us [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
</font>